## Study of housing prices in Belgium, part 2: cleaning the data

In the first part of the project we extracted all the html code for each property sold on the website. We scraped from the code all the characteristics for each of the properties. Two datasets were saved in .xls format.

In this part we merge the two datasets, and clean the data.

1. importing and merging the data sets
2. deleting the unnecessary columns/variables 
3. cleaning the variables and applying the appropriate type of each variable
4. deleting duplicated rows and otuliers/mistakes from the dataset
5. running some exploratory data analysis

### 1. Importing the datasets

In [1]:
import pandas as pd
import numpy as np
import time

date_string = "2021_11_29"
#time.strftime("%Y_%m_%d")
path = "C:/Users/Bedoret/OneDrive/Data Science/Housing prices in Belgium/jupyter_notebooks/"

pd.options.display.float_format= "{:.0f}".format
pd.options.display.max_columns = None

In [2]:
# add new data extraction to the lists

id_files = ["property_id_2021_11_29"]
char_files = ["property_char_2021_11_29"]

# extract and concatenate the IDs' files and characteristics' files
property_id = []
for file in id_files:
    data = pd.read_excel('{}{}.xls'.format(path,file), index_col=0)
    property_id.append(data)
property_id = pd.concat(property_id)   

property_char = []
for file in char_files:
    data = pd.read_excel('{}{}.xls'.format(path,file), index_col=0)
    property_char.append(data)
property_char = pd.concat(property_char)

### 2. Merging the datasets

Merging the two datasets along the ID variable which is the unique ID to each property. This variable is already defined in property_id, but is not yet defined in property_char. The unique ID of earch property is contained in the URL link of each property.

In [3]:
# extract the id values from the url string in property_char
property_char["id"] = pd.to_numeric(property_char["url"].str[-7:])

# merge the two datasets and drop all duplicates
properties = property_id.merge(property_char, how="outer", on="id")
properties.drop_duplicates(inplace = True)

# Depending on the date of the extraction, we might have to rename these variables
properties.rename(columns={'id':'Identifiant','type': 'Type', 'zip': 'Code postal', 'price': 'Prix', 'url':'URL'}, inplace=True)

### 3. Cleaning subgroups of properties

Apartment groups and house groups are properties sold as part of a group of properties. Although these could leverage valuable information on housing prices, the available data on the properties is insuficient to run valuable statistics.

In [4]:
properties = properties.loc[(properties['Type'] == "HOUSE") | 
                            (properties['Type'] == "APARTMENT")]

# uncomment to show summary of properties by type
properties.groupby("Type").mean()

,Identifiant,Prix,Code postal,Année de construction,Étage,Nombre d'étages,Nombre de façades,Surface habitable,Surface du salon,Surface de la cuisine,Chambres,Surface de la chambre 1,Surface de la chambre 2,Salles de bains,Toilettes,Surface de la terrasse,Unnamed: 23,Revenu cadastral,Site internet,Surface constructible totale au sol,Charges mensuelles,Surface de bureau,Parkings intérieurs,Parkings extérieurs,Surface de la cave,Visite virtuelle,Surface de la chambre 3,Salles de douche,Surface de la chambre 4,Surface de la chambre 5,Surface du terrain,Largeur du terrain à rue,Niveau E (performance énergétique globale),Surface du jardin,Combien de feux ouverts ?,Nombre d'annexes,Surface du grenier,Surface de l'espace professionnel,Pourcentage loué,Revenu mensuel actuel,Valeur du bien,Bouquet,Rente mensuelle,Nombre de rentiers,Âge du rentier
Type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
APARTMENT,9420102,331751,4794,1997,38,4,2,105,37,11,2,16,15,1,1,20,nan,nan,nan,200,nan,14,3,4,8,nan,14,1,14,13,nan,nan,734904,331,1,1,31,31,nan,nan,nan,nan,nan,1,72
HOUSE,9474452,433536,5424,1968,nan,nan,3,239,40,19,4,20,17,2,2,35,nan,nan,nan,289,nan,24,2,3,41,nan,17,2,16,16,1450,nan,26163,971,1,1,48,60,84,2750,nan,nan,nan,1,73


### 4. Removing unnecessary colunms

There is a bunch of colmns/variables that are not necessary for the analysis. In addition, some variables contain too litle information. We can look at the percentage of missing data for each variable.

Note: "Adresse" has been incorectly extracted from the data: it corresponds to the adress of the selling agency, not the adress of the property ! Must be deleted.

In [5]:
def percent_missing(df):
    percent_missing = pd.DataFrame(df.isnull().mean() * 100)
    percent_missing.sort_values(0, inplace = True, ascending = False)
    return percent_missing
percent_missing(properties)

,0
Largeur du terrain à rue,100
Revenu cadastral,100
Site internet,100
Valeur du bien,100
Bouquet,100
...,...
Prix,0
Type,0
URL,0
Code postal,0


In [6]:
properties = properties[["Identifiant",
                        "Type",
                        "Étage",
                        "Code postal",
                        "Prix",
                        "Surface habitable",
                        "Surface du terrain",
                        "Chambres",
                        "Type de cuisine",
                        "Salles de bains",
                        "Salles de douche",
                        "Toilettes",
                        "Terrasse",
                        "Surface de la terrasse",
                        "Jardin",
                        "Surface du jardin",
                        "Nombre de façades",
                        "Parkings extérieurs",
                        "Parkings intérieurs",
                        "Année de construction",
                        "État du bâtiment",
                        "Type de chauffage",
                        "Classe énergétique",
                        "URL"]]

### 5. Cleaning the column and applying the type for each variable

#### Cuisine, classe énergétique & bâtiment

In [7]:
# creating dictionnaries of the new labels for the variables 'type de cuisine',
# 'classe énergétique', and 'état du batiment'.

cuisine_dictionary={'Américaine hyper-équipée':1,'Hyper équipée':1,
                    'Américaine équipée':2,'Équipée':2,
                    'Américaine semi-équipée':3, 'Semi-équipée':3,
                    'Américaine non-équipée':4, 'Pas équipée':4
                    }
classe_ener_dictionary={'A++':1,
                        'A+':2,
                        'A':3,
                        'B':4,
                        'C':5,
                        'D':6,
                        'E':7,
                        'F':8,
                        'G':9,
                        'Non communiqué': None
                       }
batiment_dictionary={"Excellent état":1,
                     "Fraîchement rénové":2,
                     "Bon":3,
                     "À rafraîchir":4,
                     "À rénover":5,
                     "À restaurer":6
                    }

properties["Classe énergétique"] = pd.np.where(properties["Classe énergétique"].str.contains("_"), "Non communiqué", properties["Classe énergétique"])

properties = properties.replace({"Type de cuisine":cuisine_dictionary,"Classe énergétique":classe_ener_dictionary,"État du bâtiment":batiment_dictionary})

<ipython-input-7-ea3e256e3548>:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  properties["Classe énergétique"] = pd.np.where(properties["Classe énergétique"].str.contains("_"), "Non communiqué", properties["Classe énergétique"])


#### Terrasse, Jardin & Parkings

In [8]:
# Clean and complete some missing values

# if any terrace size has been marked, ensure that the variable terrace is marked as yes
properties['Terrasse_new'] = np.where(((properties['Terrasse'] == "Oui")| 
                                       (properties['Surface de la terrasse'] > 0)
                                      ), 1, 0
                                     )

# if any garden size has been marked, ensure that the variable garden is marked as yes
properties['Jardin_new'] = np.where(((properties['Jardin'] == "Oui")| 
                                     (properties['Surface du jardin'] > 0)
                                    ), 1, 0
                                   )

# create the variable parking as a boolean if any indoor or outdoor parking is given
properties['Parking'] = np.where(((properties['Parkings extérieurs'] > 0)| 
                                  (properties['Parkings intérieurs'] > 0)
                                 ), 1, 0
                                )

properties = properties.drop(columns=['Terrasse', 'Jardin','Parkings extérieurs','Parkings intérieurs'])
properties = properties.rename(columns= {"Terrasse_new":"Terrasse",
                                         "Jardin_new": "Jardin"})

### 6. Cleaning unnecessary rows

#### Duplicates

In [9]:
# Several properties are sold by different real estate agencies on the same website. We must delete those duplicates.
properties.drop_duplicates(subset = properties.columns.difference(["Identifiant","URL","Page"]), inplace = True)

#### Removing missing values and outliers

In [31]:
# check outliers for apartments:
properties.loc[properties["Type"]=="APARTMENT"].sort_values("Salles de bains",ascending=False).head(10)

,Identifiant,Type,Étage,Code postal,Prix,Prix m2,Surface habitable,Surface du terrain,Chambres,Type de cuisine,Salles de bains,Ratio chambres sdb,Toilettes,Terrasse,Surface de la terrasse,Jardin,Surface du jardin,Nombre de façades,Parking,Année de construction,État du bâtiment,Type de chauffage,Classe énergétique,URL
53068,9546668,APARTMENT,nan,8970,180000,450,400,nan,4,2,8,0,5,0,nan,0,nan,nan,0,1953,1,NaN,4,https://www.immoweb.be/fr/annonce/9546668
53486,9568261,APARTMENT,nan,9000,799000,6146,130,nan,6,2,6,1,6,0,nan,0,nan,3,0,1950,3,Gaz,nan,https://www.immoweb.be/fr/annonce/9568261
128,9320747,APARTMENT,nan,1000,2250000,4091,550,nan,5,1,6,1,6,1,15,0,nan,nan,1,1930,1,Gaz,5,https://www.immoweb.be/fr/annonce/9320747
15822,9585793,APARTMENT,2,2300,495000,1743,284,nan,6,2,6,1,nan,0,nan,0,nan,2,0,1998,nan,NaN,6,https://www.immoweb.be/fr/annonce/9585793
5549,8991986,APARTMENT,1,1180,3295000,6865,480,nan,5,1,5,1,4,1,nan,0,nan,4,1,1982,1,Gaz,5,https://www.immoweb.be/fr/annonce/8991986
5504,9522680,APARTMENT,1,1180,2985000,7462,400,nan,5,2,5,1,6,1,100,0,nan,4,1,1993,3,NaN,5,https://www.immoweb.be/fr/annonce/9522680
42808,8441029,APARTMENT,nan,8000,1895000,4032,470,nan,5,nan,5,1,4,0,nan,0,nan,nan,0,2002,3,NaN,4,https://www.immoweb.be/fr/annonce/8441029
23785,8721832,APARTMENT,nan,3500,399500,967,413,nan,5,nan,5,1,2,0,nan,0,nan,3,0,1982,nan,Gaz,6,https://www.immoweb.be/fr/annonce/8721832
2238,9618648,APARTMENT,nan,1050,2700000,4847,557,nan,5,1,5,1,5,1,50,1,60,2,1,1960,1,Gaz,6,https://www.immoweb.be/fr/annonce/9618648
318,9079380,APARTMENT,nan,1000,2250000,4500,500,nan,5,1,5,1,5,1,nan,0,nan,2,1,1894,2,Gaz,5,https://www.immoweb.be/fr/annonce/9079380


In [35]:
properties.sort_values("Année de construction", ascending=True).head(10)

,Identifiant,Type,Étage,Code postal,Prix,Prix m2,Surface habitable,Surface du terrain,Chambres,Type de cuisine,Salles de bains,Ratio chambres sdb,Toilettes,Terrasse,Surface de la terrasse,Jardin,Surface du jardin,Nombre de façades,Parking,Année de construction,État du bâtiment,Type de chauffage,Classe énergétique,URL
42103,9445152,HOUSE,nan,7850,280000,601,466,800,4,nan,1,4,3,1,12,1,600,2,0,1753,5,Gaz,7,https://www.immoweb.be/fr/annonce/9445152
42715,9546387,HOUSE,nan,8000,559000,2167,258,nan,2,nan,nan,nan,nan,1,nan,0,nan,nan,1,1753,1,Gaz,nan,https://www.immoweb.be/fr/annonce/9546387
42796,9635180,HOUSE,nan,8000,559000,2167,258,nan,3,2,1,3,2,1,25,0,nan,3,1,1753,3,Gaz,5,https://www.immoweb.be/fr/annonce/9635180
133,9580774,HOUSE,nan,1000,1400000,4667,300,100,4,1,1,4,3,0,nan,0,nan,2,0,1753,1,NaN,6,https://www.immoweb.be/fr/annonce/9580774
31975,9475172,HOUSE,nan,4990,248000,1102,225,614,4,4,1,4,1,1,15,0,nan,3,0,1754,nan,Mazout,7,https://www.immoweb.be/fr/annonce/9475172
27517,9412886,HOUSE,nan,4020,295000,1967,150,74,3,2,1,3,2,1,14,0,nan,2,0,1755,1,Gaz,6,https://www.immoweb.be/fr/annonce/9412886
42580,9466001,HOUSE,nan,8000,725000,2746,264,nan,5,nan,5,1,nan,1,9,1,92,2,0,1755,1,NaN,4,https://www.immoweb.be/fr/annonce/9466001
58383,9614151,HOUSE,nan,9500,359000,1632,220,205,4,2,1,4,1,1,nan,1,nan,2,0,1755,1,Gaz,4,https://www.immoweb.be/fr/annonce/9614151
42886,9456492,HOUSE,nan,8000,399000,3000,133,63,1,nan,1,1,nan,0,nan,0,nan,2,0,1755,1,Gaz,6,https://www.immoweb.be/fr/annonce/9456492
6124,6480859,HOUSE,nan,1190,599000,2304,260,nan,3,3,1,3,nan,1,nan,0,nan,2,0,1760,3,Gaz,9,https://www.immoweb.be/fr/annonce/6480859


In [33]:
# delete properties who do not refer any size, or which size is bellow 10 or above 7000
properties.drop(properties[(properties['Surface habitable'].isna()) | 
                           (properties['Surface habitable'] < 10) |
                           (properties['Surface habitable'] > 7000) 
                          ].index, inplace=True)

# delete properties which have no price or price is bellow 10 000 €
properties.drop(properties[(properties['Prix'].isna()) | 
                           (properties['Prix'] < 10000)
                          ].index, inplace=True)

# delete properties who have over 30 bedrooms: These are mostly block of appartments sold in batches 
# and should have been referred as "Appartment_Group" or "House_Group" as defined above.
# Below 30 rooms, some are batches of appartments, but some are small castle, huge villas.
properties.drop(properties[properties.Chambres > 30].index, inplace = True)

#delete APARTMENTS which have a size over 600 square meters. These are errors, or misreported 
# as apartments instead of houses, or sort of lofts/warehouses which should not be included in the analysis
properties.drop(properties[(properties['Type']=="APARTMENT") & 
                           (properties['Surface habitable']> 600)
                          ].index, inplace=True )

#delete APARTMENTS which have over 6 bedrooms. These are mainly errors, or misreported as apartments instead of houses 
# or group of apartments, or other types of properties which should not be included in the analysis.
properties.drop(properties[(properties['Type']=="APARTMENT") & 
                           (properties['Chambres']> 6) 
                          ].index, inplace=True )

# mark apartments with over 10 toilets as missing values
properties.loc[(properties['Type'] == "APARTMENT") &
               (properties['Toilettes'] >10), 
               'Toilettes'] = None

# mark properties with over 50 floors as missing values
properties.loc[properties['Étage'] > 50, 'Étage'] = None

# mark properties with less than 5 square meters area as missing values
properties.loc[properties['Surface du terrain'] <= 5, 'Surface du terrain'] = None

# mark properties with terrace size over 1000 square meter as missing values
properties.loc[properties['Surface de la terrasse'] >= 1000, 'Surface de la terrasse'] = None

# mark properties with garden size less than 5 square meters as missing values
properties.loc[properties['Surface du jardin'] <= 5, 'Surface du jardin'] = None

# mark properties who have reported over 4 facades as 4 facades
properties.loc[properties['Nombre de façades'] > 4, 'Nombre de façades'] = 4

# mark properties with construction year above 2025 as missing values
properties.loc[properties['Année de construction'] > 2025, 'Année de construction'] = None

#### Create new variables

In [36]:
# create a new variable which gives the number of bedrooms per bathroom
properties["Ratio chambres sdb"] = properties["Chambres"] / properties["Salles de bains"]

# create a variable which gives the price per square meter
properties["Prix m2"]= properties["Prix"] / properties["Surface habitable"]
properties = properties.drop(properties[properties["Prix m2"].isna()].index)

# reorder the colunms
properties = properties[["Identifiant","Type","Étage","Code postal","Prix", "Prix m2","Surface habitable","Surface du terrain","Chambres","Type de cuisine","Salles de bains","Ratio chambres sdb","Toilettes","Terrasse","Surface de la terrasse","Jardin","Surface du jardin","Nombre de façades","Parking","Année de construction","État du bâtiment","Type de chauffage","Classe énergétique","URL"]]

#### Independent drops of rows which are inappropriate

### 7. Exploratory data analysis

Let's have a look at our cleaned dataset. 
- How many properties do we have?
- How many houses/appartments?
- Average price / average price for a house vs appartment
- Average price per square meter / for houses vs appartment

In [38]:
properties.shape

(46484, 24)

In [39]:
properties.groupby("Type").mean()

,Identifiant,Étage,Code postal,Prix,Prix m2,Surface habitable,Surface du terrain,Chambres,Type de cuisine,Salles de bains,Ratio chambres sdb,Toilettes,Terrasse,Surface de la terrasse,Jardin,Surface du jardin,Nombre de façades,Parking,Année de construction,État du bâtiment,Classe énergétique
Type,,,,,,,,,,,,,,,,,,,,,
APARTMENT,9424821,3,4614,334462,3251,103,nan,2,2,1,2,1,1,19,0,358,2,0,1997,2,5
HOUSE,9481803,nan,5168,448350,2016,224,1423,4,2,2,3,2,1,33,0,1038,3,1,1966,3,6


In [40]:
houses = properties.loc[properties["Type"]=="HOUSE"]
apartments = properties.loc[properties["Type"]=="APARTMENT"]

We have 3 data bases to work with:

    - properties which contains all the data about all the properties
    
    - apartments contains data about apartments only
    
    - houses contains data about houses only
Save the 3 databases in an excel format

In [41]:
properties.to_excel('{}properties_{}.xlsx'.format(path,date_string), sheet_name= 'properties')
apartments.to_excel('{}apartments_{}.xlsx'.format(path,date_string), sheet_name= 'apartments')
houses.to_excel('{}houses_{}.xlsx'.format(path,date_string), sheet_name= 'houses')